In [ ]:
# With all variables
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
import numpy as np
from sklearn.model_selection import GridSearchCV

# define the Logistic Regression model
model = LogisticRegression()

# define a range of values for the regularization parameter C to test
param_grid = {'C': [0.01, 0.1, 1, 10, 100]}

# use grid search to find the best value of C
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1')
grid_search.fit(X_train, y_train)

# print the best value of C and the corresponding F1 score on the validation set
print("Best parameter:", grid_search.best_params_)
print("Validation set F1 score:", grid_search.best_score_)

# evaluate the performance of the model on the test set using the best parameter found
model_best = LogisticRegression(C=grid_search.best_params_['C'])
model_best.fit(X_train, y_train)
y_pred_lr = model_best.predict(X_val)

# get the classification report and confusion matrix
print(classification_report(y_val, y_pred_lr))
print(confusion_matrix(y_val, y_pred_lr))

# Generate confusion matrix plot
cm = confusion_matrix(y_val, y_pred_lr)
sns.heatmap(cm, annot=True, cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# calculate the AUC score and plot the ROC curve
y_pred_proba = model_best.predict_proba(X_val)[:,1]
auc = roc_auc_score(y_val, y_pred_proba)
fpr, tpr, thresholds = roc_curve(y_val, y_pred_proba)
plt.plot(fpr, tpr, label=f'AUC = {auc:.3f}')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()

# plot the learning curve
train_sizes, train_scores, test_scores = learning_curve(
    model_best, X_train, y_train, cv=5, n_jobs=-1,
    train_sizes=np.linspace(0.1, 1.0, 10), scoring='f1'
)
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)
plt.figure()
plt.title('Learning Curve')
plt.xlabel('Training Examples')
plt.ylabel('F1 Score')
plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.1,
                 color='r')
plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.1, color='g')
plt.plot(train_sizes, train_scores_mean, 'o-', color='r',
         label='Training F1 Score')
plt.plot(train_sizes, test_scores_mean, 'o-', color='g',
         label='Cross-Validation F1 Score')
plt.legend(loc='best')

plt.show()

##Precision recall plot

from sklearn.metrics import precision_recall_curve, average_precision_score

# calculate precision and recall for different probability thresholds
precision, recall, thresholds = precision_recall_curve(y_val, y_pred_proba)

# calculate the average precision score
average_precision = average_precision_score(y_val, y_pred_proba)

# plot the precision-recall curve
plt.plot(recall, precision, label=f'AP = {average_precision:.3f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.show()
